# Proceso ETL aplicado a MultipleChoiceResponses.csv

## Descripción del Dataset

El dataset **MultipleChoiceResponses.csv** es la encuesta anual de Machine Learning y Data Science de Kaggle del año 2019. Esta encuesta recopila información de profesionales en el campo de la ciencia de datos y machine learning a nivel mundial.

### Características del Dataset

- **Fuente**: Kaggle Machine Learning & Data Science Survey 2019
- **Formato**: Archivo CSV con respuestas de opción múltiple
- **Alcance**: Encuesta global que incluye profesionales de diferentes países
- **Propósito**: Analizar tendencias, herramientas y prácticas en el campo de ML/DS

### Entregables del Proceso ETL

Este notebook generará los siguientes archivos de salida:

1. **CSV limpio**: `/mnt/data/multipleChoiceResponses_clean.csv`
2. **Excel**: `/mnt/data/multipleChoiceResponses_clean.xlsx`
3. **Base de datos SQLite**: `/mnt/data/etl_results.db`
4. **Reporte de calidad**: `data_quality_report.json`
5. **Log de transformaciones**: `transformation_log.csv`
6. **Script Power Query**: `powerquery_replica.pq`

### Contexto Académico

Este proceso ETL se desarrolla en el contexto del curso de **Business Intelligence** de la **Universidad Peruana Unión (UPEU)**, aplicado específicamente al área de **Ingeniería de Sistemas**. El análisis de este dataset proporciona insights valiosos sobre:

- Tecnologías más utilizadas en la industria
- Tendencias salariales por región y experiencia
- Herramientas de desarrollo preferidas
- Plataformas cloud más adoptadas
- Lenguajes de programación dominantes

### Estructura del Notebook

El proceso ETL se ejecutará en las siguientes etapas:

1. **Extracción**: Carga y validación inicial del dataset
2. **Análisis Exploratorio**: Exploración de la estructura y calidad de los datos
3. **Transformación**: Limpieza, normalización y enriquecimiento de datos
4. **Carga**: Exportación a diferentes formatos para análisis
5. **Reportes**: Generación de métricas de calidad y logs
6. **Validación**: Comparación con Power BI para consistencia


## 1. Configuración del Entorno

### Importación de Librerías

Antes de comenzar el proceso ETL, es necesario importar todas las librerías requeridas. Utilizaremos las siguientes librerías estándar para el análisis de datos:

- **pandas**: Manipulación y análisis de datos estructurados
- **numpy**: Operaciones numéricas y matemáticas
- **matplotlib**: Visualizaciones básicas
- **seaborn**: Visualizaciones estadísticas avanzadas
- **sqlalchemy**: Conexión y operaciones con bases de datos
- **json**: Manejo de archivos JSON
- **datetime**: Operaciones con fechas y tiempos

### Configuración de Parámetros

Estableceremos parámetros de visualización y configuración para asegurar que los resultados sean consistentes y profesionales.


In [ ]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlalchemy
import json
import datetime
import os
import warnings

# Configuración de parámetros de visualización
plt.style.use('default')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Configuración de pandas para mostrar más información
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("✅ Librerías importadas correctamente")
print(f"📅 Fecha de ejecución: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🐍 Versión de Python: {pd.__version__}")


## 2. Extracción de Datos

### Descripción del Proceso de Extracción

La extracción de datos es la primera fase del proceso ETL (Extract, Transform, Load). En esta etapa, cargamos el dataset desde su fuente original y realizamos una validación inicial para entender su estructura y calidad.

### Origen de los Datos

El dataset **MultipleChoiceResponses.csv** proviene de la encuesta anual de Kaggle sobre Machine Learning y Data Science. Esta encuesta es una de las más comprehensivas del sector, recopilando información de miles de profesionales a nivel mundial.

### Proceso de Carga

Utilizaremos pandas para cargar el archivo CSV, especificando los parámetros apropiados para manejar la codificación y estructura del archivo. Es importante verificar que la carga se realice correctamente y que no se pierdan datos durante el proceso.


In [ ]:
# Carga del dataset
print("📊 Cargando dataset MultipleChoiceResponses.csv...")

# Intentar cargar desde la ruta especificada, si no existe, usar la ruta local
try:
    df = pd.read_csv('/mnt/data/multipleChoiceResponses.csv')
    print("✅ Dataset cargado desde /mnt/data/")
except FileNotFoundError:
    try:
        df = pd.read_csv('multipleChoiceResponses.csv')
        print("✅ Dataset cargado desde directorio local")
    except FileNotFoundError:
        print("❌ Error: No se encontró el archivo multipleChoiceResponses.csv")
        print("💡 Asegúrate de que el archivo esté en el directorio correcto")

# Información básica del dataset
print(f"\n📋 INFORMACIÓN BÁSICA DEL DATASET:")
print(f"   • Dimensiones: {df.shape[0]:,} filas × {df.shape[1]:,} columnas")
print(f"   • Memoria utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"   • Tipos de datos únicos: {df.dtypes.nunique()}")

# Mostrar las primeras filas
print(f"\n🔍 PRIMERAS 5 FILAS DEL DATASET:")
df.head()


In [ ]:
# Información detallada del dataset
print("📊 INFORMACIÓN DETALLADA DEL DATASET:")
print("="*60)

# Información general
print(f"📋 INFORMACIÓN GENERAL:")
print(f"   • Número de filas: {df.shape[0]:,}")
print(f"   • Número de columnas: {df.shape[1]:,}")
print(f"   • Total de celdas: {df.shape[0] * df.shape[1]:,}")

# Información de tipos de datos
print(f"\n🔢 TIPOS DE DATOS:")
tipos_datos = df.dtypes.value_counts()
for tipo, cantidad in tipos_datos.items():
    print(f"   • {tipo}: {cantidad} columnas")

# Información de memoria
memoria_total = df.memory_usage(deep=True).sum() / 1024**2
memoria_por_fila = memoria_total * 1024 / df.shape[0]
print(f"\n💾 USO DE MEMORIA:")
print(f"   • Memoria total: {memoria_total:.2f} MB")
print(f"   • Memoria por fila: {memoria_por_fila:.2f} KB")

# Mostrar info() completo
print(f"\n📋 INFORMACIÓN COMPLETA (df.info()):")
df.info()


## 3. Análisis Exploratorio de Datos (EDA)

### ¿Qué es el EDA?

El **Análisis Exploratorio de Datos (EDA)** es una fase crucial en cualquier proyecto de análisis de datos. Consiste en examinar los datos para entender su estructura, identificar patrones, detectar anomalías y evaluar la calidad de los datos antes de proceder con la transformación.

### Objetivos del EDA

1. **Comprensión de la estructura**: Entender cómo están organizados los datos
2. **Identificación de problemas**: Detectar valores faltantes, duplicados, inconsistencias
3. **Análisis de distribuciones**: Comprender cómo se distribuyen las variables
4. **Detección de outliers**: Identificar valores atípicos que puedan afectar el análisis
5. **Preparación para transformación**: Determinar qué limpieza y transformaciones son necesarias

### Metodología del EDA

Realizaremos un EDA sistemático que incluye:

- **Estadísticas descriptivas**: Medidas de tendencia central y dispersión
- **Análisis de completitud**: Evaluación de valores faltantes por columna
- **Detección de duplicados**: Identificación de registros repetidos
- **Análisis de tipos de datos**: Verificación de la consistencia de tipos
- **Visualizaciones exploratorias**: Gráficos para entender distribuciones


In [ ]:
# Análisis de valores faltantes
print("❌ ANÁLISIS DE VALORES FALTANTES")
print("="*50)

# Calcular valores faltantes por columna
valores_faltantes = df.isnull().sum()
porcentaje_faltantes = (valores_faltantes / len(df)) * 100

# Crear DataFrame con el análisis
analisis_faltantes = pd.DataFrame({
    'Columna': valores_faltantes.index,
    'Valores_Faltantes': valores_faltantes.values,
    'Porcentaje_Faltantes': porcentaje_faltantes.values
})

# Ordenar por número de valores faltantes
analisis_faltantes = analisis_faltantes.sort_values('Valores_Faltantes', ascending=False)

print(f"📊 RESUMEN DE VALORES FALTANTES:")
print(f"   • Total de valores faltantes: {valores_faltantes.sum():,}")
print(f"   • Columnas con valores faltantes: {(valores_faltantes > 0).sum()}")
print(f"   • Columnas completas: {(valores_faltantes == 0).sum()}")

# Mostrar las 10 columnas con más valores faltantes
print(f"\n🔝 TOP 10 COLUMNAS CON MÁS VALORES FALTANTES:")
print(analisis_faltantes.head(10).to_string(index=False))

# Categorizar columnas por nivel de completitud
completas = (porcentaje_faltantes == 0).sum()
pocos_faltantes = ((porcentaje_faltantes > 0) & (porcentaje_faltantes <= 20)).sum()
moderados_faltantes = ((porcentaje_faltantes > 20) & (porcentaje_faltantes <= 50)).sum()
muchos_faltantes = ((porcentaje_faltantes > 50) & (porcentaje_faltantes <= 80)).sum()
criticos_faltantes = (porcentaje_faltantes > 80).sum()

print(f"\n📋 CATEGORIZACIÓN POR COMPLETITUD:")
print(f"   • Completas (0% faltantes): {completas} columnas")
print(f"   • Pocos faltantes (0-20%): {pocos_faltantes} columnas")
print(f"   • Moderados (20-50%): {moderados_faltantes} columnas")
print(f"   • Muchos (50-80%): {muchos_faltantes} columnas")
print(f"   • Críticos (>80%): {criticos_faltantes} columnas")


In [ ]:
# Análisis de duplicados
print("\n🔄 ANÁLISIS DE REGISTROS DUPLICADOS")
print("="*50)

# Detectar duplicados
duplicados = df.duplicated()
num_duplicados = duplicados.sum()
porcentaje_duplicados = (num_duplicados / len(df)) * 100

print(f"📊 ANÁLISIS DE DUPLICADOS:")
print(f"   • Registros duplicados: {num_duplicados:,}")
print(f"   • Porcentaje de duplicados: {porcentaje_duplicados:.2f}%")
print(f"   • Registros únicos: {len(df) - num_duplicados:,}")

if num_duplicados > 0:
    print(f"\n🔍 EJEMPLOS DE REGISTROS DUPLICADOS:")
    # Mostrar algunos ejemplos de duplicados
    duplicados_ejemplos = df[duplicados].head(3)
    print(duplicados_ejemplos.to_string())
else:
    print(f"\n✅ No se encontraron registros duplicados")

# Análisis de valores únicos por columna
print(f"\n🔢 ANÁLISIS DE VALORES ÚNICOS POR COLUMNA")
print("-"*50)

valores_unicos = df.nunique()
valores_unicos_ordenados = valores_unicos.sort_values(ascending=False)

print(f"📊 ESTADÍSTICAS DE VALORES ÚNICOS:")
print(f"   • Promedio de valores únicos por columna: {valores_unicos.mean():.1f}")
print(f"   • Mediana de valores únicos por columna: {valores_unicos.median():.1f}")
print(f"   • Máximo de valores únicos: {valores_unicos.max()}")
print(f"   • Mínimo de valores únicos: {valores_unicos.min()}")

# Identificar columnas constantes y casi constantes
columnas_constantes = (valores_unicos == 1).sum()
columnas_casi_constantes = (valores_unicos <= 5).sum()
columnas_casi_unicas = (valores_unicos >= len(df) * 0.95).sum()

print(f"\n📋 CATEGORIZACIÓN DE COLUMNAS:")
print(f"   • Columnas constantes (1 valor único): {columnas_constantes}")
print(f"   • Columnas casi constantes (≤5 valores): {columnas_casi_constantes}")
print(f"   • Columnas casi únicas (≥95% únicos): {columnas_casi_unicas}")

# Mostrar las 10 columnas con más valores únicos
print(f"\n🔝 TOP 10 COLUMNAS CON MÁS VALORES ÚNICOS:")
top_unicos = valores_unicos_ordenados.head(10)
for col, unicos in top_unicos.items():
    print(f"   • {col}: {unicos} valores únicos")


In [ ]:
# Estadísticas descriptivas para columnas numéricas
print("\n📊 ESTADÍSTICAS DESCRIPTIVAS")
print("="*50)

# Identificar columnas numéricas
columnas_numericas = df.select_dtypes(include=[np.number]).columns
columnas_categoricas = df.select_dtypes(include=['object']).columns

print(f"📋 CLASIFICACIÓN DE COLUMNAS:")
print(f"   • Columnas numéricas: {len(columnas_numericas)}")
print(f"   • Columnas categóricas: {len(columnas_categoricas)}")

if len(columnas_numericas) > 0:
    print(f"\n🔢 ESTADÍSTICAS DESCRIPTIVAS - COLUMNAS NUMÉRICAS:")
    estadisticas_numericas = df[columnas_numericas].describe()
    print(estadisticas_numericas.to_string())
    
    # Análisis adicional para columnas numéricas
    print(f"\n📈 ANÁLISIS ADICIONAL - COLUMNAS NUMÉRICAS:")
    for col in columnas_numericas:
        valores_unicos = df[col].nunique()
        valores_faltantes = df[col].isnull().sum()
        print(f"   • {col}:")
        print(f"     - Valores únicos: {valores_unicos}")
        print(f"     - Valores faltantes: {valores_faltantes}")
        print(f"     - Rango: {df[col].min():.2f} - {df[col].max():.2f}")

# Análisis de columnas categóricas principales
print(f"\n📝 ANÁLISIS DE COLUMNAS CATEGÓRICAS PRINCIPALES")
print("-"*50)

# Analizar las primeras 5 columnas categóricas
columnas_principales = columnas_categoricas[:5]
for col in columnas_principales:
    print(f"\n🔹 {col}:")
    valores_unicos = df[col].nunique()
    valores_faltantes = df[col].isnull().sum()
    print(f"   • Valores únicos: {valores_unicos}")
    print(f"   • Valores faltantes: {valores_faltantes}")
    
    if valores_unicos <= 20:  # Mostrar distribución si no hay muchos valores únicos
        distribucion = df[col].value_counts().head(10)
        print(f"   • Top 10 valores más frecuentes:")
        for valor, count in distribucion.items():
            porcentaje = (count / len(df)) * 100
            print(f"     - {valor}: {count} ({porcentaje:.1f}%)")
    else:
        print(f"   • Demasiados valores únicos para mostrar distribución")


## 4. Transformación de Datos

### Descripción del Proceso de Transformación

La transformación de datos es la fase central del proceso ETL, donde aplicamos una serie de operaciones para limpiar, normalizar y enriquecer los datos. Esta fase es crítica para asegurar la calidad y consistencia de los datos que serán utilizados en análisis posteriores.

### Estrategias de Transformación

Basándonos en los hallazgos del EDA, aplicaremos las siguientes transformaciones:

1. **Eliminación de duplicados**: Remover registros completamente duplicados
2. **Manejo de valores nulos**: Estrategias diferenciadas según el porcentaje de valores faltantes
3. **Normalización de strings**: Limpieza de espacios, estandarización de formatos
4. **Conversión de tipos**: Optimización de tipos de datos para eficiencia
5. **Renombrado de columnas**: Mapeo de Q1, Q2, etc. a nombres descriptivos
6. **Agrupación de categorías**: Consolidación de valores poco frecuentes

### Criterios de Decisión

- **Columnas con >80% nulos**: Eliminación completa
- **Columnas con 20-80% nulos**: Imputación con valores apropiados
- **Columnas con <20% nulos**: Imputación conservadora
- **Duplicados**: Eliminación completa para evitar sesgos
- **Strings**: Normalización a minúsculas y eliminación de espacios


In [ ]:
# Inicializar log de transformaciones
transformation_log = []
timestamp_inicio = datetime.datetime.now()

print("🔄 INICIANDO PROCESO DE TRANSFORMACIÓN")
print("="*60)

# Crear copia del dataset original para transformación
df_clean = df.copy()
filas_iniciales = len(df_clean)
columnas_iniciales = len(df_clean.columns)

print(f"📊 ESTADO INICIAL:")
print(f"   • Filas: {filas_iniciales:,}")
print(f"   • Columnas: {columnas_iniciales:,}")

# PASO 1: Eliminación de duplicados
print(f"\n🔄 PASO 1: ELIMINACIÓN DE DUPLICADOS")
print("-"*40)

duplicados_antes = df_clean.duplicated().sum()
df_clean = df_clean.drop_duplicates()
duplicados_eliminados = duplicados_antes
filas_despues_duplicados = len(df_clean)

print(f"   • Duplicados encontrados: {duplicados_antes:,}")
print(f"   • Duplicados eliminados: {duplicados_eliminados:,}")
print(f"   • Filas restantes: {filas_despues_duplicados:,}")

# Registrar en log
transformation_log.append({
    'timestamp': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'operacion': 'Eliminacion_Duplicados',
    'filas_antes': filas_iniciales,
    'filas_despues': filas_despues_duplicados,
    'filas_eliminadas': duplicados_eliminados,
    'descripcion': f'Eliminados {duplicados_eliminados} registros duplicados'
})

print(f"   ✅ Duplicados eliminados exitosamente")


In [ ]:
# PASO 2: Manejo de valores nulos
print(f"\n❌ PASO 2: MANEJO DE VALORES NULOS")
print("-"*40)

# Calcular porcentaje de nulos por columna
porcentaje_nulos = (df_clean.isnull().sum() / len(df_clean)) * 100

# Categorizar columnas por nivel de nulos
columnas_eliminar = porcentaje_nulos[porcentaje_nulos > 80].index.tolist()
columnas_imputar = porcentaje_nulos[(porcentaje_nulos > 0) & (porcentaje_nulos <= 80)].index.tolist()
columnas_completas = porcentaje_nulos[porcentaje_nulos == 0].index.tolist()

print(f"📊 CATEGORIZACIÓN DE COLUMNAS:")
print(f"   • Para eliminar (>80% nulos): {len(columnas_eliminar)}")
print(f"   • Para imputar (0-80% nulos): {len(columnas_imputar)}")
print(f"   • Completas (0% nulos): {len(columnas_completas)}")

# Eliminar columnas con muchos nulos
if columnas_eliminar:
    print(f"\n🗑️ ELIMINANDO COLUMNAS CON >80% NULOS:")
    for col in columnas_eliminar[:5]:  # Mostrar primeras 5
        pct = porcentaje_nulos[col]
        print(f"   • {col}: {pct:.1f}% nulos")
    if len(columnas_eliminar) > 5:
        print(f"   • ... y {len(columnas_eliminar) - 5} columnas más")
    
    df_clean = df_clean.drop(columns=columnas_eliminar)
    print(f"   ✅ {len(columnas_eliminar)} columnas eliminadas")

# Imputar valores nulos en columnas restantes
if columnas_imputar:
    print(f"\n🔧 IMPUTANDO VALORES NULOS:")
    columnas_imputar_restantes = [col for col in columnas_imputar if col in df_clean.columns]
    
    for col in columnas_imputar_restantes:
        nulos_antes = df_clean[col].isnull().sum()
        if nulos_antes > 0:
            # Imputación categórica
            if df_clean[col].dtype == 'object':
                df_clean[col].fillna('No especificado', inplace=True)
            # Imputación numérica
            else:
                mediana = df_clean[col].median()
                df_clean[col].fillna(mediana, inplace=True)
    
    print(f"   ✅ {len(columnas_imputar_restantes)} columnas imputadas")

# Verificar nulos restantes
nulos_restantes = df_clean.isnull().sum().sum()
print(f"\n📊 RESULTADO MANEJO DE NULOS:")
print(f"   • Nulos restantes: {nulos_restantes:,}")

# Registrar en log
transformation_log.append({
    'timestamp': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'operacion': 'Manejo_Valores_Nulos',
    'filas_antes': filas_despues_duplicados,
    'filas_despues': len(df_clean),
    'filas_eliminadas': len(columnas_eliminar),
    'descripcion': f'Eliminadas {len(columnas_eliminar)} columnas, imputados nulos en {len(columnas_imputar_restantes)} columnas'
})


In [ ]:
# PASO 3: Normalización de strings
print(f"\n🔧 PASO 3: NORMALIZACIÓN DE STRINGS")
print("-"*40)

# Identificar columnas de texto
columnas_texto = df_clean.select_dtypes(include=['object']).columns
print(f"📝 COLUMNAS DE TEXTO IDENTIFICADAS: {len(columnas_texto)}")

# Normalizar strings
for col in columnas_texto:
    if col in df_clean.columns:
        # Limpiar espacios al inicio y final
        df_clean[col] = df_clean[col].astype(str).str.strip()
        # Reemplazar múltiples espacios por uno solo
        df_clean[col] = df_clean[col].str.replace(r'\s+', ' ', regex=True)
        # Convertir a minúsculas para consistencia
        df_clean[col] = df_clean[col].str.lower()

print(f"   ✅ {len(columnas_texto)} columnas de texto normalizadas")

# PASO 4: Renombrado de columnas principales
print(f"\n📝 PASO 4: RENOMBRADO DE COLUMNAS")
print("-"*40)

# Mapeo de columnas Q1-Q50 a nombres descriptivos
mapeo_columnas = {
    'Time from Start to Finish (seconds)': 'Tiempo_Total_Encuesta_Segundos',
    'Q1': 'Edad_Encuestado',
    'Q1_OTHER_TEXT': 'Edad_Encuestado_Texto_Libre',
    'Q2': 'Genero',
    'Q3': 'Pais_Residencia',
    'Q4': 'Nivel_Educativo',
    'Q5': 'Area_Estudios_Principal',
    'Q6': 'Situacion_Laboral_Actual',
    'Q6_OTHER_TEXT': 'Situacion_Laboral_Texto_Libre',
    'Q7': 'Cargo_Principal_Trabajo',
    'Q7_OTHER_TEXT': 'Cargo_Texto_Libre',
    'Q8': 'Anos_Experiencia_Campo',
    'Q9': 'Rango_Salarial_Anual',
    'Q10': 'Lenguajes_Programacion_Usados',
    'Q11_Part_1': 'IDE_Jupyter_Notebooks',
    'Q11_Part_2': 'IDE_RStudio',
    'Q11_Part_3': 'IDE_PyCharm',
    'Q11_Part_4': 'IDE_Atom',
    'Q11_Part_5': 'IDE_MATLAB',
    'Q12_MULTIPLE_CHOICE': 'Hardware_Analisis_Datos',
    'Q13_Part_1': 'Cloud_AWS',
    'Q13_Part_2': 'Cloud_Microsoft_Azure',
    'Q13_Part_3': 'Cloud_Google_Cloud',
    'Q13_Part_4': 'Cloud_IBM',
    'Q14_Part_1': 'TPU_Google',
    'Q15_Part_1': 'BigData_Spark',
    'Q15_Part_2': 'BigData_Hadoop'
}

# Aplicar renombrado solo a columnas que existen
columnas_existentes = {k: v for k, v in mapeo_columnas.items() if k in df_clean.columns}
df_clean = df_clean.rename(columns=columnas_existentes)

print(f"📝 RENOMBRADO DE COLUMNAS:")
print(f"   • Columnas renombradas: {len(columnas_existentes)}")

# Mostrar algunos ejemplos
print(f"\n🔹 EJEMPLOS DE RENOMBRADO:")
for i, (original, nuevo) in enumerate(list(columnas_existentes.items())[:8]):
    print(f"   {i+1}. {original[:35]:<35} → {nuevo}")

if len(columnas_existentes) > 8:
    print(f"   ... y {len(columnas_existentes) - 8} columnas más")

# Registrar en log
transformation_log.append({
    'timestamp': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'operacion': 'Normalizacion_y_Renombrado',
    'filas_antes': len(df_clean),
    'filas_despues': len(df_clean),
    'filas_eliminadas': 0,
    'descripcion': f'Normalizadas {len(columnas_texto)} columnas de texto, renombradas {len(columnas_existentes)} columnas'
})


In [ ]:
# PASO 5: Creación de variables derivadas
print(f"\n➕ PASO 5: CREACIÓN DE VARIABLES DERIVADAS")
print("-"*40)

# 1. Categoría de experiencia
if 'Anos_Experiencia_Campo' in df_clean.columns:
    def categorizar_experiencia(experiencia):
        if pd.isna(experiencia):
            return 'no especificado'
        exp_str = str(experiencia).lower()
        if any(x in exp_str for x in ['0-1', '< 1', 'less than 1']):
            return 'principiante (0-2 años)'
        elif any(x in exp_str for x in ['1-2', '2-3']):
            return 'principiante (0-2 años)'
        elif any(x in exp_str for x in ['3-4', '4-5']):
            return 'intermedio (3-5 años)'
        elif any(x in exp_str for x in ['5-10']):
            return 'avanzado (5-10 años)'
        elif any(x in exp_str for x in ['10-15', '15-20', '20+']):
            return 'experto (10+ años)'
        else:
            return 'no especificado'
    
    df_clean['Categoria_Experiencia'] = df_clean['Anos_Experiencia_Campo'].apply(categorizar_experiencia)
    print("   ✅ Categoria_Experiencia creada")

# 2. Categoría salarial
if 'Rango_Salarial_Anual' in df_clean.columns:
    def categorizar_salario(salario):
        if pd.isna(salario):
            return 'no especificado'
        sal_str = str(salario).lower()
        if 'not wish' in sal_str or 'do not' in sal_str:
            return 'no especificado'
        elif any(x in sal_str for x in ['0-10,000', '10,000-20,000']):
            return 'bajo (0-20k usd)'
        elif any(x in sal_str for x in ['20,000-30,000', '30,000-40,000', '40,000-50,000']):
            return 'medio (20-50k usd)'
        elif any(x in sal_str for x in ['50,000-60,000', '60,000-70,000', '70,000-80,000']):
            return 'alto (50-80k usd)'
        elif any(x in sal_str for x in ['80,000', '90,000', '100,000']):
            return 'muy alto (80-100k usd)'
        elif any(x in sal_str for x in ['125,000', '150,000', '200,000', '300,000', '400,000', '500,000']):
            return 'ejecutivo (100k+ usd)'
        else:
            return 'no especificado'
    
    df_clean['Categoria_Salarial'] = df_clean['Rango_Salarial_Anual'].apply(categorizar_salario)
    print("   ✅ Categoria_Salarial creada")

# 3. Región geográfica
if 'Pais_Residencia' in df_clean.columns:
    def categorizar_region(pais):
        if pd.isna(pais):
            return 'no especificado'
        pais_str = str(pais).lower()
        
        if any(p in pais_str for p in ['united states', 'canada', 'mexico']):
            return 'américa del norte'
        elif any(p in pais_str for p in ['brazil', 'argentina', 'colombia', 'chile', 'peru']):
            return 'américa latina'
        elif any(p in pais_str for p in ['united kingdom', 'germany', 'france', 'spain', 'italy', 'russia']):
            return 'europa'
        elif any(p in pais_str for p in ['india', 'china', 'japan', 'australia', 'singapore']):
            return 'asia-pacífico'
        else:
            return 'otros'
    
    df_clean['Region_Geografica'] = df_clean['Pais_Residencia'].apply(categorizar_region)
    print("   ✅ Region_Geografica creada")

# Mostrar distribuciones de variables derivadas
print(f"\n📊 DISTRIBUCIONES DE VARIABLES DERIVADAS:")

if 'Categoria_Experiencia' in df_clean.columns:
    dist_exp = df_clean['Categoria_Experiencia'].value_counts()
    print(f"\n🔹 Categoría de Experiencia:")
    for cat, count in dist_exp.head().items():
        pct = (count / len(df_clean)) * 100
        print(f"   • {cat}: {count:,} ({pct:.1f}%)")

if 'Categoria_Salarial' in df_clean.columns:
    dist_sal = df_clean['Categoria_Salarial'].value_counts()
    print(f"\n🔹 Categoría Salarial:")
    for cat, count in dist_sal.head().items():
        pct = (count / len(df_clean)) * 100
        print(f"   • {cat}: {count:,} ({pct:.1f}%)")

# Registrar en log
transformation_log.append({
    'timestamp': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'operacion': 'Creacion_Variables_Derivadas',
    'filas_antes': len(df_clean),
    'filas_despues': len(df_clean),
    'filas_eliminadas': 0,
    'descripcion': 'Creadas 3 variables derivadas: Categoria_Experiencia, Categoria_Salarial, Region_Geografica'
})


In [ ]:
# Resumen final de la transformación
print(f"\n📊 RESUMEN FINAL DE LA TRANSFORMACIÓN")
print("="*60)

# Métricas finales
filas_finales = len(df_clean)
columnas_finales = len(df_clean.columns)
nulos_finales = df_clean.isnull().sum().sum()
memoria_final = df_clean.memory_usage(deep=True).sum() / 1024**2

print(f"📈 COMPARACIÓN ANTES VS DESPUÉS:")
print(f"   • Filas: {filas_iniciales:,} → {filas_finales:,} ({filas_iniciales - filas_finales:+,})")
print(f"   • Columnas: {columnas_iniciales:,} → {columnas_finales:,} ({columnas_iniciales - columnas_finales:+,})")
print(f"   • Nulos: {df.isnull().sum().sum():,} → {nulos_finales:,} ({df.isnull().sum().sum() - nulos_finales:+,})")
print(f"   • Memoria: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB → {memoria_final:.1f} MB")

# Porcentajes de mejora
reduccion_nulos = ((df.isnull().sum().sum() - nulos_finales) / df.isnull().sum().sum() * 100) if df.isnull().sum().sum() > 0 else 0
reduccion_memoria = ((df.memory_usage(deep=True).sum() / 1024**2 - memoria_final) / (df.memory_usage(deep=True).sum() / 1024**2) * 100)

print(f"\n📊 MEJORAS LOGRADAS:")
print(f"   • Reducción de nulos: {reduccion_nulos:.1f}%")
print(f"   • Reducción de memoria: {reduccion_memoria:.1f}%")
print(f"   • Completitud de datos: {((filas_finales * columnas_finales - nulos_finales) / (filas_finales * columnas_finales) * 100):.1f}%")

# Resumen de transformaciones aplicadas
print(f"\n✅ TRANSFORMACIONES APLICADAS:")
print(f"   • ✅ Eliminación de duplicados")
print(f"   • ✅ Manejo inteligente de valores nulos")
print(f"   • ✅ Limpieza de espacios en blanco")
print(f"   • ✅ Normalización de datos")
print(f"   • ✅ Renombrado de columnas descriptivo")
print(f"   • ✅ Creación de variables derivadas")

# Mostrar primeras filas del dataset transformado
print(f"\n📋 PRIMERAS 3 FILAS DEL DATASET TRANSFORMADO:")
columnas_mostrar = [col for col in df_clean.columns if any(x in col for x in ['Edad', 'Genero', 'Pais', 'Nivel', 'Categoria'])][:6]

if columnas_mostrar:
    muestra = df_clean[columnas_mostrar].head(3)
    for i, (idx, row) in enumerate(muestra.iterrows()):
        print(f"\n   Fila {i+1}:")
        for col in columnas_mostrar:
            valor = str(row[col])[:30] + "..." if len(str(row[col])) > 30 else str(row[col])
            print(f"     • {col}: {valor}")

print(f"\n🎉 TRANSFORMACIÓN COMPLETADA EXITOSAMENTE")
print(f"📊 Dataset listo para carga y análisis")


## 5. Carga de Datos

### Descripción del Proceso de Carga

La carga de datos es la fase final del proceso ETL, donde exportamos los datos limpios y transformados en diferentes formatos para su posterior análisis y uso. Esta fase asegura que los datos estén disponibles en los formatos más apropiados para diferentes tipos de análisis y herramientas.

### Formatos de Exportación

Exportaremos los datos en múltiples formatos para maximizar su utilidad:

1. **CSV**: Formato universal para análisis en Python, R y otras herramientas
2. **Excel**: Formato ideal para presentaciones y análisis en herramientas de BI
3. **SQLite**: Base de datos ligera para consultas SQL y análisis relacional
4. **Reportes JSON**: Metadatos y métricas de calidad en formato estructurado
5. **Log CSV**: Registro detallado de todas las transformaciones aplicadas

### Validación de la Carga

Después de cada exportación, verificaremos que los datos se hayan guardado correctamente comparando el número de filas y columnas, y validando la integridad de los datos.


In [ ]:
# Proceso de carga de datos
print("💾 PROCESO DE CARGA DE DATOS")
print("="*60)

# Crear directorio de salida si no existe
import os
os.makedirs('/mnt/data', exist_ok=True)

# 1. Exportar a CSV
print("\n📄 EXPORTACIÓN A CSV:")
csv_filename = '/mnt/data/multipleChoiceResponses_clean.csv'

try:
    df_clean.to_csv(csv_filename, index=False, encoding='utf-8')
    csv_size = os.path.getsize(csv_filename) / 1024  # KB
    
    print(f"   ✅ CSV creado: {csv_filename}")
    print(f"   📊 Registros: {df_clean.shape[0]:,}")
    print(f"   📋 Columnas: {df_clean.shape[1]:,}")
    print(f"   💾 Tamaño: {csv_size:.1f} KB")
    
    # Verificar carga
    df_verificacion = pd.read_csv(csv_filename)
    if df_verificacion.shape == df_clean.shape:
        print(f"   ✅ Verificación exitosa: {df_verificacion.shape[0]:,} filas × {df_verificacion.shape[1]:,} columnas")
    else:
        print(f"   ❌ Error en verificación")
        
except Exception as e:
    print(f"   ❌ Error: {str(e)}")

# 2. Exportar a Excel
print("\n📊 EXPORTACIÓN A EXCEL:")
excel_filename = '/mnt/data/multipleChoiceResponses_clean.xlsx'

try:
    with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
        # Hoja principal con datos
        df_clean.to_excel(writer, sheet_name='Datos_Limpios', index=False)
        
        # Hoja con resumen
        resumen = pd.DataFrame({
            'Métrica': ['Filas', 'Columnas', 'Valores_Nulos', 'Memoria_MB'],
            'Valor': [df_clean.shape[0], df_clean.shape[1], 
                     df_clean.isnull().sum().sum(),
                     df_clean.memory_usage(deep=True).sum() / 1024**2]
        })
        resumen.to_excel(writer, sheet_name='Resumen', index=False)
    
    excel_size = os.path.getsize(excel_filename) / 1024  # KB
    print(f"   ✅ Excel creado: {excel_filename}")
    print(f"   📄 Hojas: Datos_Limpios, Resumen")
    print(f"   💾 Tamaño: {excel_size:.1f} KB")
    
except Exception as e:
    print(f"   ❌ Error Excel: {str(e)}")


In [ ]:
# 3. Exportar a SQLite
print("\n🗄️ EXPORTACIÓN A SQLITE:")
sqlite_filename = '/mnt/data/etl_results.db'

try:
    # Crear conexión a SQLite
    engine = sqlalchemy.create_engine(f'sqlite:///{sqlite_filename}')
    
    # Exportar datos principales
    df_clean.to_sql('datos_limpios', engine, if_exists='replace', index=False)
    
    # Crear tabla de metadatos
    metadatos = pd.DataFrame({
        'metrica': ['fecha_procesamiento', 'filas_originales', 'filas_finales', 'columnas_originales', 'columnas_finales'],
        'valor': [datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 
                 filas_iniciales, filas_finales, columnas_iniciales, columnas_finales]
    })
    metadatos.to_sql('metadatos', engine, if_exists='replace', index=False)
    
    # Crear tabla de log de transformaciones
    log_df = pd.DataFrame(transformation_log)
    log_df.to_sql('log_transformaciones', engine, if_exists='replace', index=False)
    
    sqlite_size = os.path.getsize(sqlite_filename) / 1024  # KB
    print(f"   ✅ SQLite creado: {sqlite_filename}")
    print(f"   📊 Tablas: datos_limpios, metadatos, log_transformaciones")
    print(f"   💾 Tamaño: {sqlite_size:.1f} KB")
    
    # Verificar carga
    with engine.connect() as conn:
        result = conn.execute(sqlalchemy.text("SELECT COUNT(*) FROM datos_limpios")).fetchone()
        print(f"   ✅ Verificación: {result[0]:,} registros en base de datos")
    
except Exception as e:
    print(f"   ❌ Error SQLite: {str(e)}")

# 4. Crear reporte de calidad JSON
print("\n📋 CREACIÓN DE REPORTE DE CALIDAD:")
json_filename = 'data_quality_report.json'

try:
    # Calcular métricas de calidad
    calidad_report = {
        'fecha_procesamiento': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'dataset_original': {
            'filas': int(filas_iniciales),
            'columnas': int(columnas_iniciales),
            'valores_nulos': int(df.isnull().sum().sum()),
            'duplicados': int(df.duplicated().sum()),
            'memoria_mb': float(df.memory_usage(deep=True).sum() / 1024**2)
        },
        'dataset_limpio': {
            'filas': int(filas_finales),
            'columnas': int(columnas_finales),
            'valores_nulos': int(nulos_finales),
            'duplicados': 0,
            'memoria_mb': float(memoria_final)
        },
        'mejoras': {
            'reduccion_nulos_porcentaje': float(reduccion_nulos),
            'reduccion_memoria_porcentaje': float(reduccion_memoria),
            'completitud_datos_porcentaje': float(((filas_finales * columnas_finales - nulos_finales) / (filas_finales * columnas_finales) * 100))
        },
        'transformaciones_aplicadas': [
            'Eliminacion_Duplicados',
            'Manejo_Valores_Nulos',
            'Normalizacion_Strings',
            'Renombrado_Columnas',
            'Creacion_Variables_Derivadas'
        ]
    }
    
    with open(json_filename, 'w', encoding='utf-8') as f:
        json.dump(calidad_report, f, indent=2, ensure_ascii=False)
    
    json_size = os.path.getsize(json_filename) / 1024  # KB
    print(f"   ✅ Reporte JSON creado: {json_filename}")
    print(f"   💾 Tamaño: {json_size:.1f} KB")
    
except Exception as e:
    print(f"   ❌ Error JSON: {str(e)}")

# 5. Crear log de transformaciones CSV
print("\n📝 CREACIÓN DE LOG DE TRANSFORMACIONES:")
log_filename = 'transformation_log.csv'

try:
    log_df = pd.DataFrame(transformation_log)
    log_df.to_csv(log_filename, index=False, encoding='utf-8')
    
    log_size = os.path.getsize(log_filename) / 1024  # KB
    print(f"   ✅ Log CSV creado: {log_filename}")
    print(f"   📊 Operaciones registradas: {len(transformation_log)}")
    print(f"   💾 Tamaño: {log_size:.1f} KB")
    
except Exception as e:
    print(f"   ❌ Error Log: {str(e)}")

print(f"\n🎉 CARGA COMPLETADA EXITOSAMENTE")
print(f"📁 Archivos listos para análisis y validación")


## 6. Comparación con Power BI (Power Query)

### Descripción de la Validación Cruzada

Para asegurar la consistencia y reproducibilidad del proceso ETL, es fundamental validar que los resultados obtenidos en Python puedan ser replicados en Power BI usando Power Query. Esta validación cruzada garantiza que el proceso sea robusto y que los datos puedan ser procesados de manera consistente en diferentes herramientas.

### Estrategia de Validación

La validación se realizará mediante:

1. **Script Power Query M**: Código que replique las transformaciones principales
2. **Métricas de Comparación**: Valores de referencia para verificar consistencia
3. **Instrucciones de Implementación**: Guía paso a paso para replicar en Power BI

### Transformaciones a Replicar

Las siguientes transformaciones serán replicadas en Power Query:

- Eliminación de duplicados
- Manejo de valores nulos
- Normalización de strings
- Renombrado de columnas principales
- Creación de variables derivadas básicas


In [ ]:
# Generar script Power Query M
print("🔧 GENERANDO SCRIPT POWER QUERY M")
print("="*60)

# Script Power Query M para replicar las transformaciones
powerquery_script = f"""
// SCRIPT POWER QUERY M - ETL KAGGLE SURVEY 2019
// Replica las transformaciones realizadas en Python
// Generado automáticamente el {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

let
    // PASO 1: Cargar datos desde CSV
    Source = Csv.Document(File.Contents("multipleChoiceResponses.csv"),
        [Delimiter=",", Columns={df.shape[1]}, Encoding=65001, QuoteStyle=QuoteStyle.Csv]),
    
    // PASO 2: Promover encabezados
    #"Promoted Headers" = Table.PromoteHeaders(Source, [PromoteAllScalars=true]),
    
    // PASO 3: Eliminar duplicados
    #"Removed Duplicates" = Table.Distinct(#"Promoted Headers"),
    
    // PASO 4: Eliminar columnas con >80% nulos
    // (Nota: Agregar columnas específicas identificadas en Python)
    #"Removed High Null Columns" = #"Removed Duplicates",
    
    // PASO 5: Reemplazar valores nulos
    #"Replaced Nulls" = Table.ReplaceValue(#"Removed High Null Columns", null, "No especificado", 
        Replacer.ReplaceValue, Table.SelectColumns(#"Removed High Null Columns", 
        Table.ColumnsOfType(#"Removed High Null Columns", {{type text}}))),
    
    // PASO 6: Normalizar strings (minúsculas y espacios)
    #"Normalized Strings" = Table.TransformColumns(#"Replaced Nulls", 
        {{"Q1", Text.Lower, type text},
         {"Q2", Text.Lower, type text},
         {"Q3", Text.Lower, type text},
         {"Q4", Text.Lower, type text},
         {"Q5", Text.Lower, type text}}),
    
    // PASO 7: Renombrar columnas principales
    #"Renamed Columns" = Table.RenameColumns(#"Normalized Strings", {{
        "Q1", "Edad_Encuestado",
        "Q2", "Genero", 
        "Q3", "Pais_Residencia",
        "Q4", "Nivel_Educativo",
        "Q5", "Area_Estudios_Principal",
        "Q6", "Situacion_Laboral_Actual",
        "Q7", "Cargo_Principal_Trabajo",
        "Q8", "Anos_Experiencia_Campo",
        "Q9", "Rango_Salarial_Anual",
        "Q10", "Lenguajes_Programacion_Usados"
    }}),
    
    // PASO 8: Crear variables derivadas
    #"Added Categoria_Experiencia" = Table.AddColumn(#"Renamed Columns", "Categoria_Experiencia", 
        each if Text.Contains(Text.Lower([Anos_Experiencia_Campo] ?? ""), "0-1") then "principiante (0-2 años)"
             else if Text.Contains(Text.Lower([Anos_Experiencia_Campo] ?? ""), "10") then "experto (10+ años)"
             else if Text.Contains(Text.Lower([Anos_Experiencia_Campo] ?? ""), "5") then "avanzado (5-10 años)"
             else "intermedio (3-5 años)"),
    
    #"Added Categoria_Salarial" = Table.AddColumn(#"Added Categoria_Experiencia", "Categoria_Salarial",
        each if Text.Contains(Text.Lower([Rango_Salarial_Anual] ?? ""), "0-10") then "bajo (0-20k usd)"
             else if Text.Contains(Text.Lower([Rango_Salarial_Anual] ?? ""), "20-50") then "medio (20-50k usd)"
             else if Text.Contains(Text.Lower([Rango_Salarial_Anual] ?? ""), "50-80") then "alto (50-80k usd)"
             else if Text.Contains(Text.Lower([Rango_Salarial_Anual] ?? ""), "100") then "ejecutivo (100k+ usd)"
             else "no especificado"),
    
    // RESULTADO FINAL
    #"Final Result" = #"Added Categoria_Salarial"
in
    #"Final Result"
"""

# Guardar script Power Query
powerquery_filename = 'powerquery_replica.pq'
try:
    with open(powerquery_filename, 'w', encoding='utf-8') as f:
        f.write(powerquery_script)
    
    powerquery_size = os.path.getsize(powerquery_filename) / 1024  # KB
    print(f"✅ Script Power Query creado: {powerquery_filename}")
    print(f"💾 Tamaño: {powerquery_size:.1f} KB")
    
except Exception as e:
    print(f"❌ Error: {str(e)}")

# Generar métricas de validación
print(f"\n📊 GENERANDO MÉTRICAS DE VALIDACIÓN")
print("-"*50)

metricas_validacion = {
    'fecha_validacion': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'metricas_python': {
        'total_registros': int(filas_finales),
        'total_columnas': int(columnas_finales),
        'valores_nulos': int(nulos_finales),
        'memoria_mb': float(memoria_final),
        'duplicados': 0
    },
    'criterios_validacion': {
        'diferencia_registros_maxima': '1%',
        'diferencia_columnas_maxima': '0',
        'valores_nulos_maximos': 0,
        'coincidencia_distribuciones_minima': '95%'
    },
    'instrucciones_validacion': [
        '1. Importar multipleChoiceResponses.csv en Power BI',
        '2. Aplicar el script M generado en powerquery_replica.pq',
        '3. Comparar métricas con valores de referencia',
        '4. Validar distribuciones de variables principales',
        '5. Generar dashboard con visualizaciones clave'
    ]
}

# Guardar métricas de validación
metricas_filename = 'metricas_validacion_powerbi.txt'
try:
    with open(metricas_filename, 'w', encoding='utf-8') as f:
        f.write("MÉTRICAS DE VALIDACIÓN - PYTHON vs POWER BI\n")
        f.write("="*60 + "\n\n")
        f.write(f"Fecha de validación: {metricas_validacion['fecha_validacion']}\n\n")
        f.write("MÉTRICAS DE REFERENCIA (PYTHON):\n")
        for metrica, valor in metricas_validacion['metricas_python'].items():
            f.write(f"• {metrica}: {valor}\n")
        f.write("\nCRITERIOS DE VALIDACIÓN EXITOSA:\n")
        for criterio, valor in metricas_validacion['criterios_validacion'].items():
            f.write(f"• {criterio}: {valor}\n")
        f.write("\nINSTRUCCIONES DE VALIDACIÓN:\n")
        for instruccion in metricas_validacion['instrucciones_validacion']:
            f.write(f"{instruccion}\n")
    
    metricas_size = os.path.getsize(metricas_filename) / 1024  # KB
    print(f"✅ Métricas de validación creadas: {metricas_filename}")
    print(f"💾 Tamaño: {metricas_size:.1f} KB")
    
except Exception as e:
    print(f"❌ Error: {str(e)}")

print(f"\n🎉 VALIDACIÓN POWER BI PREPARADA")
print(f"📁 Archivos listos para comparación cruzada")
